In [ ]:
# set correct path for imports for this notebook
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# external imports 
import logging
logging.basicConfig(level=logging.INFO)

# local imports
from src.fetch.google_reviews import fetch_google_reviews 
from backend.src.fetch.google_api import fetch_place_id, fetch_restaurant_info, Restaurant
from src.nlp.extractor_openai import extract_dishes_openai, _cached_extract_single
from src.ranking.functions import assign_rankings
from src.recommendation.recs import form_recommendations

# Clear dish extractor cache to ensure fresh model load
_cached_extract_single.cache_clear()


KeyboardInterrupt: 

In [ ]:
r1_id = fetch_place_id("funkyfish berlin")
r2_id = fetch_place_id("standard pizza west berlin")
r3_id = fetch_place_id("borschert berlin")
r4_id = fetch_place_id("Gio's Berlin") 
r5_id = fetch_place_id("hasir kreuzberg")
r6_id = fetch_place_id("paolo pinkel berlin")

INFO:src.fetch.google_place:
 Found restaurant: FunkyFisch
Kantstraße 135-136, 10625 Berlin, Deutschland
ID = ChIJU7-KvONQqEcRCmPCCvo-Fg4

INFO:src.fetch.google_place:
 Found restaurant: Standard Serious Pizza Charlottenburg
Schlüterstraße 63, 10625 Berlin, Deutschland
ID = ChIJc2uwDI1RqEcRfAsy3YxyfIY

INFO:src.fetch.google_place:
 Found restaurant: Borchardt
Französische Str. 47, 10117 Berlin, Deutschland
ID = ChIJVfEmINtRqEcRsch0pslaanQ

INFO:src.fetch.google_place:
 Found restaurant: Gio's
Dresdener Str. 16, 10999 Berlin, Deutschland
ID = ChIJ65Liv5JPqEcRodr8aFFNAvE

INFO:src.fetch.google_place:
 Found restaurant: Hasir Ocakbaşı
Adalbertstraße 12, 10999 Berlin, Deutschland
ID = ChIJeQEO3FBPqEcRuw-lu4_Kv6U

INFO:src.fetch.google_place:
 Found restaurant: Paolo Pinkel
Karl-Marx-Straße 55, 12043 Berlin, Deutschland
ID = ChIJpRECXmtPqEcRJMfokCTNvD8



In [ ]:
r1 = Restaurant(r1_id)
r4 = Restaurant(r4_id)

ValueError: Google API error: INVALID_REQUEST - Error while parsing 'fields' parameter: Unsupported field name 'websiteUri'. 

In [ ]:
funky = fetch_google_reviews(r1_id)
pizza = fetch_google_reviews(r2_id)
borch = fetch_google_reviews(r3_id)
gio = fetch_google_reviews(r4_id)
hasir = fetch_google_reviews(r5_id)
paolo = fetch_google_reviews(r6_id)

In [ ]:
%autoawait
await extract_dishes_openai(paolo, True)

In [ ]:
def assign_rankings(reviews, verbose = False):
    """
    Assigns a default ranking of -1 to all dishes in reviews 
    where the source is 'google' and the dish ranking is None.
    """
    for i, review in enumerate(reviews):
        if review.get("source") == "google" and review.get("dishes"):
            google_default_ranking = -1
            for dish in review["dishes"]:
                if dish.get("ranking") is None:
                    dish["ranking"] = google_default_ranking
                    if verbose:
                        print(f"Assigned ranking {google_default_ranking} to dish '{dish.get('name')}' in review #{i+1}")  
        elif review.get("source") == "bing" and review.get("dishes"):
            pass  # Future logic for Bing reviews can be added here



In [ ]:
rest_paolo = Restaurant(r1_id)
rest_paolo

In [ ]:
assign_rankings(paolo, True)

In [ ]:
form_recommendations(paolo)

In [ ]:
get_recommendations(r2_id)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

place_id = "ChIJVfEmINtRqEcRsch0pslaanQ"

cmd = f"""curl -X GET \
  -H "Content-Type: application/json" \
  -H "X-Goog-Api-Key: {GOOGLE_API_KEY}" \
  -H "X-Goog-FieldMask: displayName,formattedAddress,websiteUri" \
  "https://places.googleapis.com/v1/places/{place_id}"
"""


import subprocess

result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
print(result.stdout)

In [ ]:
#get more data from Google API

# set correct path for imports for this notebook
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from __future__ import annotations
import os
import requests
from typing import Any, Dict, List, Optional
from dotenv import load_dotenv
from src.normalisation.normaliser import normalise_review

# ---- Logging ----
import logging
logger = logging.getLogger(__name__)

# ---- Config ----
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

BASE_URL_FINDPLACE = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
BASE_URL_DETAILS = "https://maps.googleapis.com/maps/api/place/details/json"


def fetch_restaurant_info(place_id: str) -> Dict[str, Optional[str]]:
    """
    Retrieve basic metadata for a restaurant by Place ID.
    Returns: {"name": ..., "address": ..., "id": ...}
    """
    params = {
        "place_id": place_id,
        "fields": "url",
        "key": GOOGLE_API_KEY,
    }

    resp = requests.get(BASE_URL_DETAILS, params=params)
    data = resp.json()

    if data.get("status") != "OK":
        raise ValueError(f"Google API error: {data.get('status')} - {data.get('error_message')}")

    result = data.get("result", {})
    return result


# ---- Helper Functions (place ID is normally provided in the API call) ----
def fetch_place_id(restaurant_name: str) -> Optional[str]:
    """
    Fetch the Google Place ID for a given restaurant name.
    Returns the place_id if found, otherwise None.
    """
    params = {
        "input": restaurant_name,
        "inputtype": "textquery",
        "fields": "place_id,name,formatted_address",
        "key": GOOGLE_API_KEY,
    }
    response = requests.get(BASE_URL_FINDPLACE, params=params)
    data = response.json()

    candidates = data.get("candidates", [])
    if not candidates:
        logger.warning(f"No candidates found for '{restaurant_name}'")
        return None

    candidate = candidates[0]
    logger.info(
        f"\n Found restaurant: {candidate.get('name')}\n"
        f"{candidate.get('formatted_address')}\n"
        f"ID = {candidate.get('place_id')}\n"
        f"URL = {candidate.get('websiteUri')}\n"

    )
    return candidate.get("place_id")


##
##
##

# ---- Importing packages ----

import requests

# ---- Config ----
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
BASE_URL_GOOGLEPLACES = "https://places.googleapis.com/v1/places/"

def fetch_google_places_data(place_id: str):
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": GOOGLE_API_KEY,
        "X-Goog-FieldMask": "*"
    }
    resp = requests.get(BASE_URL_GOOGLEPLACES+place_id, headers=headers)
    data = resp.json()
    
    restaurant_info = ""
    reviews = ""

    return restaurant_info, reviews


fetch_google_places_data("ChIJVfEmINtRqEcRsch0pslaanQ")

('', '')

In [1]:
# ---- Importing packages ----
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import requests
import logging
from dotenv import load_dotenv
from src.normalisation.schema import Restaurant, Review

logging.basicConfig(level=logging.INFO) #remove later in .py
logger = logging.getLogger(__name__)

# ---- Config ----
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
BASE_URL_GOOGLEPLACES = "https://places.googleapis.com/v1/places/"

def fetch_google_places_data(place_id: str):
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": GOOGLE_API_KEY,
        "X-Goog-FieldMask": (
            "id,"
            "displayName.text,"
            "formattedAddress,"
            "websiteUri,"
            "googleMapsUri,"
            "rating,"
            "reviews.rating,"
            "reviews.text.text,"
            "reviews.authorAttribution.displayName,"
            "reviews.publishTime,"
            "reviews.googleMapsUri,"
            "reviews.originalText.languageCode"
        ),
        
    }
    resp = requests.get(BASE_URL_GOOGLEPLACES+place_id, headers=headers)
    data = resp.json()
    
    logger.info(
        f"\nFound restaurant: {data.get('displayName').get('text')}\n"
        f"{data.get('formattedAddress')}\n")

    #pass data into BaseModel classes to normalise them

    reviews_data = data.get("reviews") or []
    reviews = [Review(**r) for r in reviews_data]

    restaurant = Restaurant(**data)


    return restaurant, reviews


fetch_google_places_data("ChIJVfEmINtRqEcRsch0pslaanQ")

INFO:__main__:
Found restaurant: Borchardt
Französische Str. 47, 10117 Berlin, Germany



(Restaurant(place_id='ChIJVfEmINtRqEcRsch0pslaanQ', name='Borchardt', address='Französische Str. 47, 10117 Berlin, Germany', website_url=HttpUrl('http://www.borchardt-restaurant.de/'), google_maps_url=HttpUrl('https://maps.google.com/?cid=8388617078058567857&g_mp=CiVnb29nbGUubWFwcy5wbGFjZXMudjEuUGxhY2VzLkdldFBsYWNlEAIYBCAA'), rating=4.1),
 [Review(author='Y T', url=HttpUrl('https://www.google.com/maps/reviews/data=!4m6!14m5!1m4!2m3!1sCi9DQUlRQUNvZENodHljRjlvT2tOeFdqVkhXRXBMZDJKeFEzaE1Za1IwV1hnek1tYxAB!2m1!1s0x47a851db2026f155:0x746a5ac9a674c8b1'), rating=4.0, text='We made a reservation at this German restaurant for lunch. The service was okay, but our main dishes were left out initially.\n\nThe tartar and schnitzel were delicious, and the portions were generous. However, the seasonal menu of the deer was average.', timestamp=1760791579, original_lang='en'),
  Review(author='Олександра Карасьова', url=HttpUrl('https://www.google.com/maps/reviews/data=!4m6!14m5!1m4!2m3!1sCi9DQUlRQUNvZEN

In [1]:
# ---- Importing packages ----
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import logging
logging.basicConfig(level=logging.INFO) #remove later in .py
from src.fetch.google_api import fetch_google_places_data


fetch_google_places_data("ChIJVfEmINtRqEcRsch0pslaanQ")


INFO:src.fetch.google_api:
Found restaurant: Borchardt
Französische Str. 47, 10117 Berlin, Germany

INFO:src.fetch.google_api:
Retrieved 5 reviews for Borchardt ✅


(Restaurant(place_id='ChIJVfEmINtRqEcRsch0pslaanQ', name='Borchardt', address='Französische Str. 47, 10117 Berlin, Germany', website_url=HttpUrl('http://www.borchardt-restaurant.de/'), google_maps_url=HttpUrl('https://maps.google.com/?cid=8388617078058567857&g_mp=CiVnb29nbGUubWFwcy5wbGFjZXMudjEuUGxhY2VzLkdldFBsYWNlEAIYBCAA'), rating=4.1),
 [Review(author='Y T', url=HttpUrl('https://www.google.com/maps/reviews/data=!4m6!14m5!1m4!2m3!1sCi9DQUlRQUNvZENodHljRjlvT2tOeFdqVkhXRXBMZDJKeFEzaE1Za1IwV1hnek1tYxAB!2m1!1s0x47a851db2026f155:0x746a5ac9a674c8b1'), rating=4.0, text='We made a reservation at this German restaurant for lunch. The service was okay, but our main dishes were left out initially.\n\nThe tartar and schnitzel were delicious, and the portions were generous. However, the seasonal menu of the deer was average.', timestamp=1760791579, original_lang='en'),
  Review(author='Олександра Карасьова', url=HttpUrl('https://www.google.com/maps/reviews/data=!4m6!14m5!1m4!2m3!1sCi9DQUlRQUNvZEN